##Using CLIP to generate CLIP vectors for Image-MemeText Pairs

In [ ]:
!pip install --upgrade jupyter_http_over_ws>=0.0.7 && jupyter serverextension enable --py jupyter_http_over_ws

Enabling: jupyter_http_over_ws
- Writing config: /root/.jupyter
    - Validating...
      jupyter_http_over_ws  OK


In [ ]:
#!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-uepm2_ly
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-uepm2_ly
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=713ea91751443543bc578d61649367604663e79b638c1074d9c847cab37c2545
  Stored in directory: /tmp/pip-ephem-wheel-cache-epprq6c7/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)


Torch version: 1.12.1+cu113


In [ ]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
clip_model, preprocess = clip.load("ViT-B/32")
clip_model.cuda().eval()

100%|███████████████████████████████████████| 338M/338M [00:10<00:00, 35.3MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [1]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np, pandas as pd
np.random.seed(42)
from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Data/

Mounted at /content/drive
/content/drive/My Drive/Data


In [4]:
all_images = [filename for filename in os.listdir('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/')]

In [8]:
pd.DataFrame(all_images).to_csv('TrainDataOrder.csv', index=False)

In [ ]:
IMAGES_DIR = '/content/drive/My Drive/Data/memotion_dataset_7k/en_images/'

labels_df_A = pd.read_csv('/content/drive/My Drive/Data/memotion_dataset_7k/Labels_A.csv')
labels_df_A.head()

descriptions = {}
for row in labels_df_A.index:
  descriptions[labels_df_A.image_name.tolist()[row]] = labels_df_A.text_corrected.tolist()[row]

print(descriptions)

{'image_1.jpg': 'LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com', 'image_2.jpeg': 'The best of #10 YearChallenge! Completed in less the 4 years. Kudus to @narendramodi ji 8:05 PM - 16 Jan 2019 from Mumbai  India', 'image_3.JPG': "Sam Thorne @Strippin ( Follow Follow Saw everyone posting these 2009 vs 2019 pics so here's mine 6:23 PM - 12 Jan 2019 O 636 Retweets 3 224 LIKES 65 636 3.2K", 'image_5.png': '10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious 10 Year Challenge Memes | What is #10 Year Challenge?', 'image_6.jpg': '1998: "Don\'t get in car with strangers" 2008: "Don\'t meet people from the internet alone." MemeCenter.com 2019: UBER.. Order yourself a stranger from the internet to get into a car with alone.', 'image_9.jpg': 'Fornite died in 10 months but Minecraft never died in 10 years I am so proud of this community', 'image_10.png': "FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMPLE MEME? mo by eXo Hạ Minh Le 18 Jan 2018",

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
original_images = []
images = []
texts = []

for filename in [filename for filename in os.listdir('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/')][:3000]:
  name = os.path.splitext(filename)[0]
  print(os.path.join('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/', filename))
  image = Image.open(os.path.join('/content/drive/My Drive/Data/memotion_dataset_7k/en_images/', filename)).convert("RGB")
  print(name, descriptions[filename])
  original_images.append(image)
  images.append(preprocess(image))
  texts.append(descriptions[filename])

/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_5750.jpg
image_5750 Can I have a light show Brother? @bassnectoe
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_5751.jpg
image_5751 I AM THE BEST JOKER EVER WHY YOU SO SERIOUS
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_5752.jpg
image_5752 Covers @ FirstCovers.com IF YOU THINK I CAN'T KICK YOUR ASS UP AND DOWN THIS FLOOR
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_5753.png
image_5753 Expert in muggle technology Can't pronounce 'electricity' ROFLB01 joyreactor.com This always annoyed me for some reason... ~Moonyo~
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_5754.png
image_5754 FLASHBACK: Trump said about Kim Jong-un: "He speaks and his people sit up in attention. I want my people to do the same." Unbelievable. This dangerous moron wants to be America's first tyrant OCCUPY DEMOCRATS
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/ima

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


image_6513 THE JOKER JUST CALLED.  HE WANTS HIS GRIN BACK!
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6514.png
image_6514 YOU WAKE UP AS THE JOKER Facebook/officialmvdc WHATS THE FIRST THING YOU DO? ~Andy
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6516.jpg
image_6516 Alert: Kim Jong-un has slathered his nukes in BBQ sauce and is now threatening to send the world to Flavortown adam.the.creator P78-075
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6517.jpg
image_6517 WE ROCK! LET'S SEE HILTON BUILD SOMETHING BETTER THAN THIS ROOM imgflip.com
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6518.jpeg
image_6518 The world is really run by Dennis Rodman.
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6520.png
image_6520 BBQ SOLD OUT?! IT'S BARELY NOON!
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6522.jpg
image_6522 AND THIS IS WHERE YOU SELECT DEEP PAN OR CLASSIC CRUST We Kn

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6681.jpg
image_6681 Don't try to tell me Obama wasn't the greatest President ever Show the whole tiddy tho
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6682.jpg
image_6682 SO YOU'RE JUST GONNA LEAVE US HERE WITH THAT ANGRY CHEETO?! Olama Memes
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6684.jpg
image_6684 First off Fwek Melania & the clique she claim  he husband hair so thin I can see his brain
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6685.jpg
image_6685 The only job the GOP created in 4 years!
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6686.jpg
image_6686 PRESDENT OBAMA REACHING OUT TO FEMALE VOTERS POLITICALMEMES.BLOGSPOT.COM
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6689.jpg
image_6689 Give me your lunch money!
/content/drive/My Drive/Data/memotion_dataset_7k/en_images/image_6690.png
image_6690 RUSSIA LEADS IN

In [ ]:
len(images)

3000

In [ ]:
indices = [x for x in range(0, len(images)+1, 2)]
print(indices)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370, 372, 374, 376, 378, 380, 382, 384, 386, 388, 390, 392, 394, 396, 398, 400, 402, 404, 406, 408, 410, 412, 414, 416, 418, 420,

In [ ]:
mt = torch.empty((1,1024), dtype=torch.int64).cuda()
mt

tensor([[                0,          11119392,              7040,
          ..., 15525211561994265, 15525211561994265,
         15525211561994265]], device='cuda:0')

In [ ]:
for i in range(0,len(images),2):
  image_input = torch.tensor(np.stack(images[i:i+2])).cuda()
  text_tokens = clip.tokenize([desc for desc in texts[i:i+2]], truncate=True).cuda()
  #print(image_input.shape)
  #print(text_tokens.shape)
  with torch.no_grad():
    image_features = clip_model.encode_image(image_input).float()
    #print(image_features.shape)
    text_features = clip_model.encode_text(text_tokens).float()
    #print(text_features.shape)
  
  concat_features = torch.cat((image_features, text_features), 1)
  mt = torch.cat((mt, concat_features), 0)
  print(mt.shape)

torch.Size([3, 1024])
torch.Size([5, 1024])
torch.Size([7, 1024])
torch.Size([9, 1024])
torch.Size([11, 1024])
torch.Size([13, 1024])
torch.Size([15, 1024])
torch.Size([17, 1024])
torch.Size([19, 1024])
torch.Size([21, 1024])
torch.Size([23, 1024])
torch.Size([25, 1024])
torch.Size([27, 1024])
torch.Size([29, 1024])
torch.Size([31, 1024])
torch.Size([33, 1024])
torch.Size([35, 1024])
torch.Size([37, 1024])
torch.Size([39, 1024])
torch.Size([41, 1024])
torch.Size([43, 1024])
torch.Size([45, 1024])
torch.Size([47, 1024])
torch.Size([49, 1024])
torch.Size([51, 1024])
torch.Size([53, 1024])
torch.Size([55, 1024])
torch.Size([57, 1024])
torch.Size([59, 1024])
torch.Size([61, 1024])
torch.Size([63, 1024])
torch.Size([65, 1024])
torch.Size([67, 1024])
torch.Size([69, 1024])
torch.Size([71, 1024])
torch.Size([73, 1024])
torch.Size([75, 1024])
torch.Size([77, 1024])
torch.Size([79, 1024])
torch.Size([81, 1024])
torch.Size([83, 1024])
torch.Size([85, 1024])
torch.Size([87, 1024])
torch.Size([89,

In [ ]:
row_exclude = 1
mt = torch.cat((mt[:row_exclude],mt[row_exclude+1:]))
mt.shape

torch.Size([3000, 1024])

In [ ]:
torch.save(mt, 'tensor_0_3000.pt')

In [ ]:
mt_0 = torch.load('tensor_0_3000.pt')
mt_1 = torch.load('tensor_1_3000.pt')
mt_0.shape, mt_1.shape

(torch.Size([3000, 1024]), torch.Size([2544, 1024]))

In [ ]:
mt_fin = torch.cat((mt_0, mt_1), 0)
mt_fin.shape

torch.Size([5544, 1024])

In [ ]:
torch.save(mt_fin, 'tensor_fin.pt')

In [ ]:
IMAGES_DIR = '/content/drive/My Drive/Data/test_data/en_test_images/'

labels_df_A = pd.read_csv('/content/drive/My Drive/Data/test_data/Labels_test_A.csv')
labels_df_A.head()

descriptions = {}
for row in labels_df_A.index:
  descriptions[labels_df_A.image_name.tolist()[row]] = labels_df_A.text_corrected.tolist()[row]

print(descriptions)

{'chuck_chuck_norris_meme_10.jpg': 'Some magicians can walk on water  Chuck Norris can swim through land.', 'misog_2109e457d636565e2e06dce39874c5231e1.jpg': "Me: Mom can my friend sleep over? Mom: That's fine but no boys!! Me: #GrowingUpGay", 'obama_2691536739_469698809820026_263513986_n.jpg': 'THIS GUY INHERITED A MESS. DID HE WHINE ABOUT IT? NO HE FOXED THINGS. THIS GUY...NOT SO MUCH . FORAMERICA', 'kim_threat-kim-jong-un-allegedly-working-on-multiple-attack-missiles-36036632.png': "THREAT: Kim Jong Un allegedly working on multiple attack missiles wee woo wee woo wee woo.. IG:PolarSaurusRex Update: reports say Kim Jong Un's missiles could have been deployed DEWO So what year do y'all think ww3 will take place", 'big_bang_cfe5c1d4d28c66694e01996f6ed2e70765b442c958e94178ee32f0b47d8497ec.jpg': 'YOU CANT-RUIN A FRIENDSHIP WITH SEX  THATS LIKE TRYING TO RUIN ICECREAM WITH SPRINKLE Q & A', 'misog_217images-12.jpg': 'GOES ON A DATE WITH OVERLY ATTACHED GIRLFRIEND', 'bean_21stv8.jpg': '2018 

In [ ]:
original_images = []
images = []
texts = []

for filename in [filename for filename in os.listdir(IMAGES_DIR)]:
  name = os.path.splitext(filename)[0]
  print(os.path.join(IMAGES_DIR, filename))
  image = Image.open(os.path.join(IMAGES_DIR, filename)).convert("RGB")
  print(name, descriptions[filename])
  original_images.append(image)
  images.append(preprocess(image))
  texts.append(descriptions[filename])

/content/drive/My Drive/Data/test_data/en_test_images/harvey_4082494.jpg
harvey_4082494 I'M YOUR MAN.STEVE HARVEY! HAPPY MEMORIAL DAY EVERYBODY! OH! MY BAD! HAPPY INDEPENDENCE DAY!
/content/drive/My Drive/Data/test_data/en_test_images/feminist_tequilafunrise3.jpg
feminist_tequilafunrise3 when I feel any kind of pain AESTHETICIZE IT AND PUT IT ONLINE FOR EASY CONSUMPTION
/content/drive/My Drive/Data/test_data/en_test_images/chuck_955fd3b0fcf8d5b1475eb0b3d4f48dbc0ccfc121345b2a9ec8ea26083bfc1bd9.jpg
chuck_955fd3b0fcf8d5b1475eb0b3d4f48dbc0ccfc121345b2a9ec8ea26083bfc1bd9 CHUCK NORRIS BEATEN BY A POSTER OF BRUCE LEE
/content/drive/My Drive/Data/test_data/en_test_images/country_club-giggles-memes-that-are-sweeping-the-country-13191.jpg
country_club-giggles-memes-that-are-sweeping-the-country-13191 YO MAMA SO NASTY SHE PUT SALTWATER DOWN HER PANTS TO KEEP HER CRABS FRESH
/content/drive/My Drive/Data/test_data/en_test_images/liam_30bg37.jpg
liam_30bg37 WHEN YOUR MOM'S GOING THROUGH YOUR PHONE B

In [ ]:
mt = torch.empty((1,1024), dtype=torch.int64).cuda()
mt

tensor([[5665587200,       6656,          0,  ...,          1,  128707536,
          128707520]], device='cuda:0')

In [ ]:
for i in range(0,len(images),2):
  image_input = torch.tensor(np.stack(images[i:i+2])).cuda()
  text_tokens = clip.tokenize([desc for desc in texts[i:i+2]], truncate=True).cuda()
  #print(image_input.shape)
  #print(text_tokens.shape)
  with torch.no_grad():
    image_features = clip_model.encode_image(image_input).float()
    #print(image_features.shape)
    text_features = clip_model.encode_text(text_tokens).float()
    #print(text_features.shape)
  
  concat_features = torch.cat((image_features, text_features), 1) #columns concatenation
  mt = torch.cat((mt, concat_features), 0) #row concatenation
  print(mt.shape)

torch.Size([3, 1024])
torch.Size([5, 1024])
torch.Size([7, 1024])
torch.Size([9, 1024])
torch.Size([11, 1024])
torch.Size([13, 1024])
torch.Size([15, 1024])
torch.Size([17, 1024])
torch.Size([19, 1024])
torch.Size([21, 1024])
torch.Size([23, 1024])
torch.Size([25, 1024])
torch.Size([27, 1024])
torch.Size([29, 1024])
torch.Size([31, 1024])
torch.Size([33, 1024])
torch.Size([35, 1024])
torch.Size([37, 1024])
torch.Size([39, 1024])
torch.Size([41, 1024])
torch.Size([43, 1024])
torch.Size([45, 1024])
torch.Size([47, 1024])
torch.Size([49, 1024])
torch.Size([51, 1024])
torch.Size([53, 1024])
torch.Size([55, 1024])
torch.Size([57, 1024])
torch.Size([59, 1024])
torch.Size([61, 1024])
torch.Size([63, 1024])
torch.Size([65, 1024])
torch.Size([67, 1024])
torch.Size([69, 1024])
torch.Size([71, 1024])
torch.Size([73, 1024])
torch.Size([75, 1024])
torch.Size([77, 1024])
torch.Size([79, 1024])
torch.Size([81, 1024])
torch.Size([83, 1024])
torch.Size([85, 1024])
torch.Size([87, 1024])
torch.Size([89,

In [ ]:
row_exclude = 1
mt = torch.cat((mt[:row_exclude],mt[row_exclude+1:]))
mt.shape

torch.Size([1496, 1024])

In [ ]:
torch.save(mt, 'tensor_fin_test.pt')

In [ ]:
dir(mt[0:3])

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__cuda_array_interface__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__

In [ ]:
mt[0:3].cpu().numpy().flatten()

array([ 5.6655872e+09,  6.6560000e+03,  0.0000000e+00, ...,
        6.8750000e-01,  2.3742676e-01, -1.3623047e-01], dtype=float32)

In [ ]:
mt[0:3].cpu().numpy().flatten().shape

(3072,)